In [1]:
import sys
sys.path.append('../mmdetection/') # add mmdetection to sys path
import glob
import pandas as pd
from mmcv import Config # should pip install mmcv first
from mmdet.models import build_detector
from mmdet.utils import get_model_complexity_info

def show_complexity(config, input_shape=(3,1280,800)):
    print(config)
    cfg = Config.fromfile(config)
    model = build_detector(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg).cuda()
    model.eval()

    if hasattr(model, 'forward_dummy'):
        model.forward = model.forward_dummy
    else:
        raise NotImplementedError(
            'FLOPs counter is currently not currently supported with {}'.
            format(model.__class__.__name__))

    flops, params = get_model_complexity_info(model, input_shape)    
    df = pd.DataFrame([[config.split("/")[-1], flops, params]], columns=["Config", "nFlops", "nParams"])
    return df

def show_all_complexity(paths):
    black_list = ["caffe", "dpool", "gcnet", "ms_rcnn", "mask_rcnn"]  # do not search for config contain those key words
    all_complexity = pd.DataFrame(columns=["Config", "nFlops", "nParams"])
    for path in paths:
        if any(s in path for s in black_list):
            continue
        all_complexity = all_complexity.append(show_complexity(path))
        
    return all_complexity
    
# change the below code to the absolute path to configs folder, "~" is not working here
other_paths = glob.glob("/home/wedward.wei/code/mmdetection/configs/*/*.py")
root_paths = glob.glob("/home/wedward.wei/code/mmdetection/configs/*.py")
paths = root_paths + other_paths

In [ ]:
all_complexity = show_all_complexity(paths)

In [ ]:
all_complexity_other = show_all_complexity(other_paths) # result is folded in this notebook, since it is too long

In [ ]:
show_complexity("~/code/mmdetection/configs/faster_rcnn_x101_32x4d_fpn_1x.py")

In [9]:
pd.set_option('display.max_row', None)
all_complexity.sort_values(by='Config').reset_index(drop=True)

,Config,nFlops,nParams
0,cascade_rcnn_dconv_c3-c5_r50_fpn_1x.py,206.61 GMac,69.75 M
1,cascade_rcnn_hrnetv2p_w32_20e.py,313.05 GMac,74.92 M
2,cascade_rcnn_r101_fpn_1x.py,310.78 GMac,88.16 M
3,cascade_rcnn_r50_fpn_1x.py,234.71 GMac,69.17 M
4,cascade_rcnn_x101_32x4d_fpn_1x.py,314.53 GMac,87.8 M
5,cascade_rcnn_x101_64x4d_fpn_1x.py,468.0 GMac,126.89 M
6,dh_faster_rcnn_r50_fpn_1x.py,481.27 GMac,47.12 M
7,fast_rcnn_r101_fpn_1x.py,232.5 GMac,59.93 M
8,fast_rcnn_r50_fpn_1x.py,156.43 GMac,40.94 M
9,faster_rcnn_dconv_c3-c5_r50_fpn_1x.py,178.97 GMac,42.11 M


In [10]:
all_complexity.sort_values(by='nFlops', ascending=False).reset_index(drop=True)

,Config,nFlops,nParams
0,htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fp...,724.86 GMac,142.96 M
1,htc_x101_64x4d_fpn_20e_16gpu.py,720.98 GMac,137.82 M
2,htc_x101_32x4d_fpn_20e_16gpu.py,567.51 GMac,98.73 M
3,htc_hrnetv2p_w32_20e.py,566.03 GMac,85.85 M
4,htc_r101_fpn_20e.py,563.76 GMac,99.09 M
5,htc_r50_fpn_1x.py,487.69 GMac,80.1 M
6,htc_r50_fpn_20e.py,487.69 GMac,80.1 M
7,dh_faster_rcnn_r50_fpn_1x.py,481.27 GMac,47.12 M
8,retinanet_x101_64x4d_fpn_1x.py,472.61 GMac,95.46 M
9,cascade_rcnn_x101_64x4d_fpn_1x.py,468.0 GMac,126.89 M
